<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.4: Sequential Logic
**Prev: [Control Flow](2.3_control_flow.ipynb)**<br>
**Next: [FIR Filter](2.5_exercise.ipynb)**

## Motivation
You can't write any meaningful digital logic without state. You can't write any meaningful digital logic without state. You can't write any meaningful digital logic....

Get it? Because without storing intermediate results, you can't get anywhere.

Ok, that bad joke aside, this module will describe how to express common sequential patterns in Chisel. By the end of the module, you should be able to implement and test a shift register in Chisel.

It's important to emphasize that this section will probably not dramatically impress you. Chisel's power is not in new sequential logic patterns, but in the parameterization of a design. Before we demonstrate that capability, we have to learn what these sequential patterns are. Thus, this section will show you that Chisel can do pretty much what Verilog can do - you just need to learn the Chisel syntax.

## Setup

In [5]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/dan/dev/dan-fritchman/chisel-bootcamp/source/load-ivy.sc"

In [6]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# Registers
The basic stateful element in Chisel is the register, denoted `Reg`.
A `Reg` holds its output value until the rising edge of its clock, at which time it takes on the value of its input.
By default, every Chisel `Module` has an implicit clock that is used by every register in the design.
This saves you from always specifying the same clock all over your code.

<span style="color:blue">**Example: Using a Register**</span><br>
The following code block implements a module that takes the input, adds 1 to it, and connects it as the input of a register.
*Note: The implicit clock can be overridden for multi-clock designs. See the appendix for an example.*

In [7]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = Reg(UInt(12.W))
  register := io.in + 1.U
  io.out := register
}

class RegisterModuleTester(c: RegisterModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegisterModule) { c => new RegisterModuleTester(c) })
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.153] Done elaborating.
Total FIRRTL Compile Time: 65.6 ms
Total FIRRTL Compile Time: 42.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574711897633
test RegisterModule Success: 100 tests passed in 105 cycles taking 0.048856 seconds
[info] [0.047] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegisterModule
defined class RegisterModuleTester

The register is created by calling `Reg(tpe)`, where `tpe` is a variable that encodes the type of register we want.
In this example, `tpe` is a 12-bit `UInt`.

Look at what the tester above is doing.
Between calls to `poke()` and `expect`, there is a call to `step(1)`.
This tells the test harness to tick the clock once, which will cause the register to pass its input to its output.

Calling `step(n)` will tick the clock `n` times.

The astute observer will notice that previous testers testing combinational logic did not call `step()`. This is because calling `poke()` on an input immediately propagates the updated values through combinational logic. Calling `step()` is only needed to update state elements in sequential logic. 

The code block below will show the verilog generated by `RegisterModule`.

Note:
* The module has an input for clock (and reset) that you didn't add- this is the implicit clock
* The variable `register` shows up as `reg [11:0]`, as expected
* There is a block sectioned off by ` `ifdef Randomize` that initialized the register to some random variable before simulation starts
* `register` is updated on `posedge clock`

In [8]:
println(getVerilog(new RegisterModule))

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 114.1 ms
module RegisterModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] register; // @[cmd6.sc 7:21]
  reg [31:0] _RAND_0;
  assign io_out = register; // @[cmd6.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  register = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  `en

One important note is that Chisel distinguishes between types (like `UInt`) and hardware nodes (like the literal `2.U`, or the output of `myReg`). While
```scala
val myReg = Reg(UInt(2.W))
```
is legal because a Reg needs a data type as a model,
```scala
val myReg = Reg(2.U)
```
is an error because `2.U` is already a hardware node and can't be used as a model.

<span style="color:blue">**Example: RegNext**</span><br>
Chisel has a convenience register object for registers with simple input connections. The previous `Module` can be shortened to the following `Module`. Notice how we didn't need to specify the register bitwidth this time. It gets inferred from the register's output connection, in this case `io.out`.

In [9]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

class RegNextModuleTester(c: RegNextModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegNextModule) { c => new RegNextModuleTester(c) })
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.022] Done elaborating.
Total FIRRTL Compile Time: 63.8 ms
Total FIRRTL Compile Time: 33.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574712844704
test RegNextModule Success: 100 tests passed in 105 cycles taking 0.046059 seconds
[info] [0.047] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegNextModule
defined class RegNextModuleTester

The Verilog looks almost the same as before, though the register name is generated instead of explicity defined.

In [10]:
println(getVerilog(new RegNextModule))

[info] [0.000] Elaborating design...
[info] [0.067] Done elaborating.
Total FIRRTL Compile Time: 56.1 ms
module RegNextModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] _T_2; // @[cmd8.sc 8:20]
  reg [31:0] _RAND_0;
  assign io_out = _T_2; // @[cmd8.sc 8:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  _T_2 = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  `endif // RANDOMI

---
# `RegInit`

The register in `RegisterModule` was initialized to random data for simulation.
Unless otherwised specified, registers do not have a reset value (or a reset).
The way to create a register that resets to a given value is with `RegInit`.

For instance, a 12-bit register initialized to zero can be created with the following.
Both versions below are valid and do the same thing:
```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```

The first version has two arguments.
The first argument is a type node that specified the datatype and its width.
The second argument is a hardware node that specified the reset value, in this case 0.

The second version has one argument.
It is a hardware node that specifies the reset value, but normally `0.U`.

<span style="color:blue">**Example: Initialized Register** </span><br>
The following demonstrates using `RegInit()`, initialized to zero.

In [11]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

println(getVerilog(new RegInitModule))

[info] [0.000] Elaborating design...
[info] [0.037] Done elaborating.
Total FIRRTL Compile Time: 55.9 ms
module RegInitModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] register; // @[cmd10.sc 7:25]
  reg [31:0] _RAND_0;
  wire [11:0] _T_1; // @[cmd10.sc 8:21]
  assign _T_1 = io_in + 12'h1; // @[cmd10.sc 8:21]
  assign io_out = register; // @[cmd10.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT

defined class RegInitModule

Note that the generated verilog now has a block that checks `if (reset)` to reset the register to 0.
Also note that this is inside the `always @(posedge clock)` block.
Chisel's implicit reset is active high and synchronous.
The register is still initialized to random junk before reset is called.
The `PeekPokeTesters` always call reset before running your test, but you can manually call reset as well using the `reset(n)` function, where reset is high for `n` cycles.

---
# Control Flow
Registers are very similar to wires in terms of control flow.
They have last connect semantics and can be assigned to conditionally with `when`, `elsewhen`, and `otherwise`.

<span style="color:blue">**Example: Register Control Flow**</span><br>
The following example finds the maximum value in a sequence of inputs using conditional register assignments.

In [12]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}
assert(chisel3.iotesters.Driver(() => new FindMax) {
  c => new PeekPokeTester(c) {
    expect(c.io.max, 0)
    poke(c.io.in, 1)
    step(1)
    expect(c.io.max, 1)
    poke(c.io.in, 3)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 2)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 24)
    step(1)
    expect(c.io.max, 24)
  }
})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.039] Done elaborating.
Total FIRRTL Compile Time: 54.8 ms
Total FIRRTL Compile Time: 49.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574713150316
test FindMax Success: 5 tests passed in 9 cycles taking 0.008982 seconds
[info] [0.008] RAN 4 CYCLES PASSED
SUCCESS!!


defined class FindMax

---
# Other Register Examples

Operations called on a register are performed on the **output** of the register, and the kind of operations depend on the register's type.
That means that you can write
```scala
val reg: UInt = Reg(UInt(4.W))```
which means the value `reg` is of type `UInt` and you can do things you can normally do with `UInt`s, like `+`, `-`, etc.


You aren't restricted to using `UInt`s with registers, you can use any subclass of the base type `chisel3.Data`. This includes `SInt` for signed integers and a lot of other things.

<span style="color:blue">**Example: Comb Filter**</span><br>
The following example shows a comb filter.

In [13]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

[info] [0.000] Elaborating design...
[info] [0.025] Done elaborating.
Total FIRRTL Compile Time: 62.1 ms
module Comb(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] delay; // @[cmd12.sc 7:24]
  reg [31:0] _RAND_0;
  wire [11:0] _T_1; // @[cmd12.sc 9:19]
  assign _T_1 = $signed(io_in) - $signed(delay); // @[cmd12.sc 9:19]
  assign io_out = $signed(_T_1); // @[cmd12.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMI

defined class Comb

---
# Exercises
<span style="color:red">**Exercise: Shift Register**</span><br>
Given your new-found registering knowledge, build a module that implements a shift register for a LFSR. Specifically:
- Each element is a single bit wide.
- Has a 4-bit output signal.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- **The output initializes at `b0001`.**
- Shifts each clock cycle (no enable signal).
- **Note in Chisel, subword assignment IS ILLEGAL**; something like `out(0) := in` will not work.

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [16]:
class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)
    state := state << 1 | io.in
    io.out := state

}

class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    poke(c.io.in, i % 2)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    step(1)
    expect(c.io.out, state)
  }
}
assert(chisel3.iotesters.Driver(() => new MyShiftRegister()) {
    c => new MyShiftRegisterTester(c)
})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.056] Done elaborating.
Total FIRRTL Compile Time: 26.5 ms
Total FIRRTL Compile Time: 31.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574713509815
test MyShiftRegister Success: 10 tests passed in 15 cycles taking 0.011509 seconds
[info] [0.011] RAN 10 CYCLES PASSED
SUCCESS!!


defined class MyShiftRegister
defined class MyShiftRegisterTester

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Shift Register (Optional)**</span><br>
Write a shift register that is parameterized by its delay (`n`), its initial value (`init`), and also has an enable input signal (`en`).

In [19]:
// n is the output width (number of delays - 1)
// init state to init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))

when(io.en){  state := state << 1 | io.in}
    io.out := state
}

class MyOptionalShiftRegisterTester(c: MyOptionalShiftRegister) extends PeekPokeTester(c) {
  val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
  var state = c.init
  var i = 0
  poke(c.io.en, 1)
  while (i < 10 * c.n) {
    // poke in repeated inSeq
    val toPoke = inSeq(i % inSeq.length)
    poke(c.io.in, toPoke)
    // update expected state
    state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
    step(1)
    expect(c.io.out, state)

    i += 1
  }
}

// test different depths
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  assert(chisel3.iotesters.Driver(() => new MyOptionalShiftRegister(n = i)) {
    c => new MyOptionalShiftRegisterTester(c)
  })
}
println("SUCCESS!!")

Testing n=3
[info] [0.000] Elaborating design...
[info] [0.080] Done elaborating.
Total FIRRTL Compile Time: 38.7 ms
Total FIRRTL Compile Time: 26.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574713866172
test MyOptionalShiftRegister Success: 30 tests passed in 35 cycles taking 0.008835 seconds
[info] [0.011] RAN 30 CYCLES PASSED
Testing n=4
[info] [0.001] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 44.8 ms
Total FIRRTL Compile Time: 29.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574713866389
test MyOptionalShiftRegister Success: 40 tests passed in 45 cycles taking 0.016893 seconds
[info] [0.018] RAN 40 CYCLES PASSED
Testing n=8
[info] [0.000] Elaborating design...
[info] [0.011] Done elaborating.
Total FIRRTL Compile Time: 41.2 ms
Total FIRRTL Compile Time: 33.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1574713866574
test MyOptionalShiftRegister Success: 80 tests pas

defined class MyOptionalShiftRegister
defined class MyOptionalShiftRegisterTester

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := state
</pre></article></div></section></div>

---
# Appendix: Explicit clock and reset
Chisel modules have a default clock and reset that are implicitly used by every register created inside them.
There are times where you want to be able to override this default behavior; perhaps you have a black box that generates a clock or reset signal, or you have a multi-clock design.

Chisel provides constructs for dealing with these cases.
Clocks and resets can be overridden separately or together with `withClock() {}`, `withReset() {}`, and `withClockAndReset() {}`.
The following code blocks will give examples of using these functions.

One thing to be aware of is that `reset` (as of this tutorial's writing) is always synchronous and of type `Bool`.
Clocks have their own type in Chisel (`Clock`) and should be declared as such.
*`Bool`s can be converted to `Clock`s by calling `asClock()` on them, but you should be careful that you aren't doing something silly.*

Also note that `chisel-testers` do not currently have complete support for multi-clock designs.

<span style="color:blue">**Example: Multi-Clock Module**</span><br>
A module with multiple clocks and reset signals.

In [20]:
// we need to import multi-clock features
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    // everything in this scope with have alternateReset as the reset
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

[info] [0.000] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 49.6 ms
module ClockExamples(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  input        io_alternateReset,
  input        io_alternateClock,
  output [9:0] io_outImplicit,
  output [9:0] io_outAlternateReset,
  output [9:0] io_outAlternateClock,
  output [9:0] io_outAlternateBoth
);
  reg [9:0] imp; // @[cmd19.sc 14:20]
  reg [31:0] _RAND_0;
  reg [9:0] _T; // @[cmd19.sc 20:25]
  reg [31:0] _RAND_1;
  reg [9:0] _T_1; // @[cmd19.sc 26:25]
  reg [31:0] _RAND_2;
  reg [9:0] _T_2; // @[cmd19.sc 32:22]
  reg [31:0] _RAND_3;
  assign io_outImplicit = imp; // @[cmd19.sc 16:18]
  assign io_outAlternateReset = _T; // @[cmd19.sc 22:26]
  assign io_outAlternateClock = _T_1; // @[cmd19.sc 28:26]
  assign io_outAlternateBoth = _T_2; // @[cmd19.sc 34:25]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZ

import chisel3.experimental.{withClock, withReset, withClockAndReset}


defined class ClockExamples

---
# Wrap Up
Great job completing this section!! You've now learned how to create registers and write sequential logic in Chisel, which means you have enough basic building blocks to write real circuits.

The next section will combine everything we've learned into one example! If you need a little more encouragement, just remember these words from an expert Chisel user:

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# You're done!

[Return to the top.](#top)